In [66]:
import numpy as np
def data(n, m, theta):
    beta = np.random.randn(m+1,1)
    X = np.random.randn(n, m)
    x0 = np.ones((n,1))
    X = np.hstack((x0,X))
    
    exp = 1/(1+np.exp(-(np.dot(X, beta)))) 
    # print(exp)
    noise = np.random.binomial(1, theta, n)
    noise = np.reshape(noise, (n,1))
    # print("Hi Im Noise \n",noise)
    exp_noise = exp+noise
    # print("Hi I'm Exp Noise \n",exp_noise)
    y = np.where(exp_noise > .5, 1, 0) 
    return X, y, beta

In [67]:
def sigmoid(z):
    return 1/(1+ np.exp(-z))

In [68]:
def loss(pred_y, y):
    return (-y.T*np.log(pred_y) - (1-y).T*np.log(1 - pred_y)).mean()

In [69]:
def gradient(X, y, pred_y):
    return (np.matmul(X.T,(pred_y - y)))/y.shape[0]

In [70]:
def logistic(X, y, k, threshold, LR):
    rows = X.shape[0]
    colum = X.shape[1]
    
    beta = np.random.rand(colum,1)
    init_cost = float("inf")
    for _ in range(k):
        z = np.matmul(X, beta)   
        pred_y = sigmoid(z)      
        cost = loss(pred_y, y)
        if abs(init_cost - cost) <= threshold:
            break
        else:
            init_cost=cost
            grad = gradient(X,y,pred_y)
            beta=beta - (float(LR)*grad)
    return init_cost,beta

In [ ]:
X,y,w=data(10,10,0.1)
print(y)

In [85]:
cost,s = logistic(X,y,20000,0.0000004,.001)
print(cost)

0.7114323029388264


In [86]:
def loss_l1(pred_y, y, n, lmda, beta):
    return (-y.T*np.log(pred_y) - (1-y).T*np.log(1 - pred_y)).mean() + (lmda/n)*np.sum(np.absolute(beta))

def gradient_l1(X,y,pred_y,n,lmda,beta):
    g_0 = np.array(np.mean(pred_y-y)).reshape(1,1)
    new_lmda = lmda*(np.absolute(beta)/beta) 
    g_1 = (np.matmul(X.T[1:],(pred_y-y))/n) +(new_lmda/n)
    grad = np.concatenate((g_0,g_1),axis=0) 
    return grad

In [87]:
def logistic_with_l1(X, y, k, threshold, LR, lmda):
    rows = X.shape[0]
    colum = X.shape[1]
    
    beta = np.random.rand(colum,1)
    init_cost = float("inf")
    for _ in range(k):
        z = np.matmul(X, beta)            
        pred_y = sigmoid(z)               
        cost = loss_l1(pred_y, y, rows, lmda, beta[1:])
        if abs(init_cost - cost) <= threshold:
            break
        else:
            init_cost=cost
            grad = gradient_l1(X,y,pred_y, rows, lmda, beta[1:])
            beta=beta - (float(LR)*grad)
    return init_cost,beta

In [88]:
cost,s = logistic_with_l1(X,y,10000,0.0000005,.001,0.7)
print(cost)

0.8477136334983164


In [89]:
def loss_l2(pred_y, y, n, lmda, beta):
    return (-y.T*np.log(pred_y) - (1-y).T*np.log(1 - pred_y)).mean() + (lmda/n)*np.sum(beta*beta)

def gradient_l2(X,y,pred_y,n,lmda,beta):
    g_0 = np.array(np.mean(pred_y-y)).reshape(1,1)  
    g_1 = (np.matmul(X.T[1:],(pred_y-y))/n) + 2*(lmda*beta)/n              
    grad = np.concatenate((g_0,g_1),axis=0) 
    return grad

def logistic_with_l2(X, y, k, threshold, LR, lmda):
    rows = X.shape[0]
    colum = X.shape[1]
    
    beta = np.random.rand(colum,1)
    init_cost = float("inf")
    for _ in range(k):
        z = np.matmul(X, beta)         
        pred_y = sigmoid(z)            
        cost = loss_l2(pred_y, y, rows, lmda, beta[1:])
        if abs(init_cost - cost) <= threshold:
            break
        else:
            init_cost=cost   
            grad = gradient_l2(X,y,pred_y, rows, lmda, beta[1:])
            beta=beta - (float(LR)*grad)
    return init_cost,beta

In [90]:
cost,s=logistic_with_l2(X,y,10000,0.0000005,.001,0.7)
print(cost)

0.7671850804154664


In [91]:
import numpy as np
class Regression:
    def __init__(self, modal_type, regulizer=None, lmda=None):
        self.modal_type = modal_type
        self.regulizer = regulizer
        self.lmda = lmda
    
    def data(self, n, m, sigma=None, theta=None):
        if(self.modal_type=="Linear"):
            e = np.random.normal(0, sigma, n)  
            e = np.reshape(e, (n,1)) 
            beta=np.random.rand(m+1,1)    
 
            X = np.random.rand(n, m+1)    
            X[:, 0] = 1                  
            y=np.matmul(X,beta)+e          
            return X, y, beta
        
        elif(self.modal_type=="Logistic"):
   
            beta = np.random.randn(m+1,1)
            
            X = np.random.randn(n, m)
            x0 = np.ones((n,1))
            X = np.hstack((x0,X))
            exp = 1/(1+np.exp(-(np.dot(X, beta))))         
            noise = np.random.binomial(1, theta, n)   
            noise = np.reshape(noise, (n,1))
            exp_noise = exp+noise
            y = np.where(exp_noise > .5, 1, 0)

            return X, y, beta
        else:
            print("choose modal_type: Linear, Logistic")
            
    def regression(self, X, y,k, threshold):
        learning_rate = 0.001

        n = X.shape[0]
        m = X.shape[1]

        weight = np.random.rand(m,1) * learning_rate

        initial_cost = float('inf') 
        for _ in range(k):
            y_predicted = np.matmul(X, weight)
            cost = (1/n) * np.sum((y_predicted - y)**2)  
            if abs(initial_cost - cost) <= threshold:
                break
            initial_cost = cost             
            gd = (-2/n) * np.matmul((y - y_predicted).transpose(),X)   
            weight = weight - learning_rate * gd    
        return initial_cost, weight
    
    def sigmoid(self, z):
        return 1/(1+ np.exp(-z))
    
    def loss(self, pred_y, y):
        return (-y.T*np.log(pred_y) - (1-y).T*np.log(1 - pred_y)).mean()
    
    def gradient(self, X, y, pred_y):
        return (np.matmul(X.T,(pred_y - y)))/y.shape[0]
    

    def loss_l1(self, pred_y, y, n, lmda, beta):
        return (-y.T*np.log(pred_y) - (1-y).T*np.log(1 - pred_y)).mean() + (lmda/n)*np.sum(abs(beta))

    def gradient_l1(self, X,y,pred_y,n,lmda,beta):
        g_0 = np.array(np.mean(pred_y-y)).reshape(1,1)
        new_lmda = lmda*(np.absolute(beta)/beta) 
        g_1 = np.matmul(X.T[1:],(pred_y-y))/n +(new_lmda/n)
        grad = np.concatenate((g_0,g_1),axis=0)    
        return grad
        
    def loss_l2(self, pred_y, y, n, lmda, beta):
        return (-y.T*np.log(pred_y) - (1-y).T*np.log(1 - pred_y)).mean() + (lmda/(2*n))*np.sum(beta*beta)

    def gradient_l2(self, X, y, pred_y, n, lmda, beta):
        g_0 = np.array(np.mean(pred_y-y)).reshape(1,1)  
        g_1 = np.matmul(X.T[1:],(pred_y-y))/n + (lmda*beta)/n              
        grad = np.concatenate((g_0,g_1),axis=0)    
        return grad

    def logistic(self, X, y, k, threshold, LR):
        rows = X.shape[0]
        colum = X.shape[1]
    
        beta = np.random.rand(colum,1)
        init_cost = float("inf")
        for _ in range(k):
            z = np.matmul(X, beta)              
            pred_y = self.sigmoid(z)                  
            if self.regulizer=='l1':
                cost = self.loss_l1(pred_y, y, rows, self.lmda, beta[1:])
            elif self.regulizer=='l2':
                cost = self.loss_l2(pred_y, y, rows, self.lmda, beta[1:])
            else:
                cost = self.loss(pred_y, y)
            if abs(init_cost - cost) <= threshold:
                break
            else:
                init_cost=cost
                if self.regulizer=='l1':
                    grad = self.gradient_L1(X,y,pred_y, rows, self.lmda, beta[1:])
                elif self.regulizer=='l2':
                    grad = self.gradient_l2(X,y,pred_y, rows, self.lmda, beta[1:])
                else:
                    grad = self.gradient(X,y,pred_y)
                beta = beta - (float(LR)*grad)
        return init_cost,beta

In [92]:
modal = Regression(modal_type="Logistic")

In [93]:
X, y, beta = modal.data(4,4,theta=0.7)

In [94]:
cost,s = modal.logistic(X, y, 10000, 0.0000005, .001)
print(cost)

0.8251207999963981


In [95]:
modal = Regression(modal_type="Logistic", regulizer="L1", lmda=0.7)
cost, s = modal.logistic(X, y, 10000, 0.0000005, .001)
print(cost)

0.7827304365063905


In [96]:
modal = Regression(modal_type="Logistic", regulizer="l2", lmda=0.7)
cost,s = modal.logistic(X, y, 10000, 0.0000005, .001)
print(cost)

0.7809364656118876
